<a href="https://colab.research.google.com/github/DCharles01/Jigsaw-Colabs/blob/main/5-prefect-deployments/lesson/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prefect Deployment

### Introduction

So far we have seen how to get set up with prefect, and build our first flow and tasks.  However, we have not yet seen how to *schedule* this flow so that it is called say every hour.  To do this, we'll need to work with prefect **deployments**.  Ok, let's get going.

### Starting with our Flow

Ok, if you look at the current codebase, we have a flow that's defined in our `index.py` file, and the `get_restaurants` function is our flow function.

```python
import requests
from prefect import flow, task

@task
def find_receipts(name):
    url = "https://data.texas.gov/resource/naix-2893.json"
    response = requests.get(url, params = {'taxpayer_name': name})
    return response.json()[:1]

@flow
def get_restaurants(url: str):
    receipts = find_receipts(url)
    return receipts

name = 'HONDURAS MAYA CAFE & BAR LLC'
print(get_restaurants(name))
```

From here, it's a good idea to run our flow, by running our script.

`python3 index.py`

Now, because our flow has a different name then we used previously, running this flow will have created a new flow in our prefect dashboard.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/get-restaurants.png?raw=1" width="80%">

And if we click on that new flow, `get-restaurants`, we'll see the our 1 flow run.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/flow-runs.png?raw=1" width="70%">

We'll also see a tab for `Deployments`, but if we click on the tab we won't find any deployments associated with the flow.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/empty-deployments.png?raw=1" width="50%">

### Creating a deployment

Ok, so now let's create a deployment, which will allow us to schedule our flow.  

The deployment will contain the rules specifying how often, and how to run our flow.

We can create our deployment by commenting out the line that calls our flow (at the bottom of `index.py`), and adding the following to the bottom of the file:

```python
#index.py

# get_restaurants("HONDURAS MAYA CAFE & BAR LLC")

if __name__ == "__main__":
    get_restaurants.serve(name="get-restaurants-deployment")
```

Ok so if you run this script again, in your terminal, you will see a line saying that `Prefect is now looking for any scheduled runs`.  This means that prefect has created your deployment.

If you go to the prefect UI, you can see our new deployment by clicking on the deployments tab.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/deployments-tab.png?raw=1">

And if you go back to flows, and click on our get-restaurants flow, you'll now see the deployment listed in the tab.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/flow-deployment.png?raw=1" width="60%">

Now, we'll eventually get to scheduling these deployments to run automatically, but it's a good idea to first kick off our deployment manually to make it easier to debug.  

As prefect pointed out, we can kick off our deployment run with the following.

`prefect deployment run 'get-restaurants/get-restaurants-deployment'`

If you go to the prefect ui, and click on our deployment followed by runs, you'll see that this failed.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/deployment-runs.png?raw=1" width="60%">

And if you click on that run (above `ochre-rooster`), you can see the error message.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/error-msg.png?raw=1" width="90%">

The issue is that we did not tell our deployment to pass through a `url` when calling our flow.  And that argument is required.

So let's update our `serve` function (and thus our deployment) to the following:

```python
if __name__ == "__main__":
    get_restaurants.serve(
        name="get-restaurants-deployment",
        parameters={'url': "HONDURAS MAYA CAFE & BAR LLC"}
        )
```

Press `ctl + c` in the terminal, and then run the python script again.  Then kickoff another deployment run in the terminal with the following:

`prefect deployment run 'get-restaurants/get-restaurants-deployment'`

This time, if you look at the new deployment-run, you should see that it successfully completed.

### Scheduling our deployments

Ok, so now it's time to schedule our deployments.  We can do so by importing the `IntervalSchedule` class at the top of our flow.

```python
from prefect.server.schemas.schedules import IntervalSchedule
```

And then update our serve argument to be the following:

```python
if __name__ == "__main__":
    get_restaurants.serve(
        name="get-restaurants-deployment",
        schedule=IntervalSchedule(interval=100),
        parameters={'url': "HONDURAS MAYA CAFE & BAR LLC"}
        )
```

This will have our deployment run every 1 minute and 40 seconds.

And to successfully update the deployment, we now need to shut down the deployment by pressing `ctl + c` in our terminal, and then re-run our python script.

This time, if we re-click on the deployment, we'll see that it has a schedule of every 1 minute 40 seconds.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/minute-forty.png?raw=1" width="70%">

And if you click on our deployment again, and sort the flow runs from oldest to newest, we will eventually see our deployment run successfully.

<img src="https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/lesson/oldest-newest.png?raw=1" width="70%">

We'll also see that there are a lot of future deployments scheduled -- but if we press ctl+c in our terminal, these future deployments will disappear.

### Summary

In this lesson, we learned about creating and scheduling our deployments.  We created a deployment by calling the `.serve` method on our deployment.

```python
if __name__ == "__main__":
    get_restaurants.serve(name="get-restaurants-deployment")
```

And from there, we added arguments to pass parameters to our flow, and to schedule the execution of our flow.

```python
if __name__ == "__main__":
    get_restaurants.serve(
        name="get-restaurants-deployment",
        schedule=IntervalSchedule(interval=100),
        parameters={'url': "HONDURAS MAYA CAFE & BAR LLC"}
        )
```

Running the updated code will both create and schedule our deployments.